## Torch+Ignite CIFAR10

In [1]:
import torch
print("pytorch ",torch.version.__version__)
import torchvision
print("torchvision ",torchvision.version.__version__)
import ignite
print("ignite ",ignite.__version__)

pytorch  2.4.0
torchvision  0.19.0


C:\Users\hsn\scoop\apps\mambaforge\current\envs\pytorch\Lib\site-packages\ignite\handlers\checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


ignite  0.5.1


Libraries used:
1. PyTorch - [Main](https://pytorch.org/) / [conda channel](https://anaconda.org/pytorch/repo/files)
2. Pytorch Ignite - https://pytorch-ignite.ai/
3. Torchvision - https://pytorch.org/vision/stable/index.html

### Load dataset

In [22]:
# Define transformations for the training and test sets
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Custom transformation to scale normalized values back to unit [0, 1]
class ToUnitRange:
    def __call__(self, tensor):
        return (tensor + 1) / 2

# Define transformations for the training and test sets
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.0,), (1.0,))  # Normalize to [0, 1]
])

# Example usage with CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader= torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=1)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=1)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
# Define CNN model

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
# Ignite training loop

from ignite.engine import create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.engine import Events

trainer = create_supervised_trainer(model, optimizer, criterion)
# Create the evaluator
metrics = {
    'accuracy': Accuracy(),
    'loss': Loss(criterion)
}
evaluator = create_supervised_evaluator(model, metrics=metrics)

@trainer.on(Events.EPOCH_COMPLETED)
def run_validation():
    evaluator.run(testloader)
    metrics = evaluator.state.metrics
    print(f"Epoch: {trainer.state.epoch},  Accuracy: {metrics['accuracy']}")

trainer.run(trainloader, 1)